Use the tweets for the updated vocab, test classifer on humor

In [1]:
!pwd

/home/farzin/haha_2019


In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
from fastai.text import *
from fastai.callbacks import SaveModelCallback
import sentencepiece as spm #https://github.com/google/sentencepiece

# Python code Examples (testing out tokenizer)

## Data

### Reading the texts

In [ ]:
PATH = Path('./data/rnn/')

In [ ]:
all_texts_df = pd.read_csv('./data/haha_2019_train.csv')

In [ ]:
all_texts_df.head()

,id,text,is_humor,votes_no,votes_1,votes_2,votes_3,votes_4,votes_5,funniness_average
0,705196579758583809,Niveles de retraso mental: \r\n\r\n— Bajo.\r\n...,1,1,2,2,0,0,0,1.5
1,678040651817213952,"—Vamos Luke desenfunda tu sable, demuestra tu ...",1,1,3,0,1,0,0,1.5
2,546750892213829633,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",1,0,2,1,0,1,1,2.6
3,965807211292364801,No se porqué me hago la cabeza deooos,0,3,0,0,0,0,0,NaN
4,638403841839484928,Quisiera saber que hago durante la siesta de l...,0,4,0,1,0,0,0,NaN


### apply pre-rules to text

In [ ]:
## New Line char for replacement in text
NL = 'xxnl'
defaults.text_spec_tok.append(NL) #add a New Line special char

def sub_nl(t:str) -> str:
    "Replaces \n by xxnl"
    return t.replace("\r\n","\n").replace("\n",NL+" ")

# def sub_br(t:str) -> str:
#     "Replaces the <br /> by \n"
#     re_br = re.compile(r'<\s*br\s*/?>', re.IGNORECASE)
#     return re_br.sub("\n", t)

def spec_add_spaces(t:str) -> str:
    "Add spaces between special characters"
    return re.sub(r'([/#?!@,])', r' \1 ', t)

def rm_useless_spaces(t:str) -> str:
    "Remove multiple spaces"
    return re.sub(' {2,}', ' ', t)

def replace_rep(t:str) -> str:
    "Replace repetitions at the character level"
    def _replace_rep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_REP} {len(cc)+1} {c} '
    re_rep = re.compile(r'(\S)(\1{3,})')
    return re_rep.sub(_replace_rep, t)
    
def replace_wrep(t:str) -> str:
    "Replace word repetitions"
    def _replace_wrep(m:Collection[str]) -> str:
        c,cc = m.groups()
        return f' {TK_WREP} {len(cc.split())+1} {c} '
    re_wrep = re.compile(r'(\b\w+\W+)(\1{3,})')
    return re_wrep.sub(_replace_wrep, t)

def deal_caps(t:str) -> str:
    "Replace words in all caps"
    res = []
    for s in re.findall(r'\w+|\W+', t):
        res += ([f' {TK_UP} ',s.lower()] if (s.isupper() and (len(s)>2)) else [s.lower()])
    return ''.join(res)

def fixup(x:str) -> str:
    "List of replacements from html strings"
    re1 = re.compile(r'  +')
    x = x.replace('#39;', "'").replace('amp;', '&').replace('#146;', "'").replace(
        'nbsp;', ' ').replace('#36;', '$').replace('\\n', "\n").replace('quot;', "'").replace(
        '<br />', "\n").replace('\\"', '"').replace('<unk>',UNK).replace(' @.@ ','.').replace(
        ' @-@ ','-').replace('\\', ' \\ ')
    return re1.sub(' ', html.unescape(x))

default_rules = [fixup, replace_rep, replace_wrep, deal_caps, spec_add_spaces, 
                 rm_useless_spaces, sub_nl]

In [ ]:
## apply the rules
raw_text = all_texts_df.loc[:,'text']

print("Default Rules:\n",[x.__name__ for x in default_rules],"\n\n")

for rule in default_rules:
    print(rule)
    raw_text = raw_text.apply(lambda x: rule(str(x)))
    
all_texts_df['new_text'] = 'xxbos ' + raw_text

Default Rules:
 ['fixup', 'replace_rep', 'replace_wrep', 'deal_caps', 'spec_add_spaces', 'rm_useless_spaces', 'sub_nl'] 


<function fixup at 0x7f76b32b62f0>
<function replace_rep at 0x7f76b32b6158>
<function replace_wrep at 0x7f76b32b61e0>
<function deal_caps at 0x7f76b32b6268>


In [ ]:
all_texts_df['new_text'].head()

In [ ]:
all_texts_df.shape

### Clean up the imbalanced data to balance

In [ ]:
print('Number of positive samples:', (all_texts_df.is_humor == 1).sum())
print('Number of negative samples:',  (all_texts_df.is_humor == 0).sum())
print('Total samples:', len(all_texts_df))

In [ ]:
rnd_seed = 20190313
np.random.seed(rnd_seed)

idx = np.random.permutation(len(all_texts_df))
test_cut = int(0.15 * len(idx))
valid_cut = int(0.15 * len(idx-test_cut))

df_train_all = all_texts_df.iloc[idx[:-(valid_cut+test_cut)],:]
df_valid     = all_texts_df.iloc[idx[-(valid_cut+test_cut):-test_cut],:]
df_test      = all_texts_df.iloc[idx[-test_cut:],:]

In [ ]:
print('Test prevalence(n = %d):'%len(df_test),df_test.is_humor.sum()/ len(df_test))
print('Valid prevalence(n = %d):'%len(df_valid),df_valid.is_humor.sum()/ len(df_valid))
print('Train all prevalence(n = %d):'%len(df_train_all), df_train_all.is_humor.sum()/ len(df_train_all))
print('all samples (n = %d)'%len(all_texts_df))
assert len(all_texts_df) == (len(df_test)+len(df_valid)+len(df_train_all)),'math didnt work'

In [ ]:
## oversample the less-frequent occuring cases
# split the training data into positive and negative
rows_pos = df_train_all.is_humor == 1
df_train_pos = df_train_all.loc[rows_pos]
df_train_neg = df_train_all.loc[~rows_pos]
df_train_pos.shape,df_train_neg.shape

In [ ]:
df_resample_pos = df_train_pos.sample(n=len(df_train_neg),replace=True,
                                      axis=0,random_state=rnd_seed).reset_index(drop=True)
df_train = pd.concat([df_resample_pos,df_train_neg],axis=0) #randomized again in DataBunch?
print('Train prevalence (n = %d):'%len(df_train), df_train.is_humor.sum()/ len(df_train))

### sp tokenizer from training

In [ ]:
model_prefix = './all_tweets_es_0509'

In [ ]:
## itos from m.vocab file: just read directly and populate the dictionary
itos = [] #{}
with open(f'{model_prefix}.vocab','r') as f:
    for line_num,line in enumerate(f):
#         itos[line_num] = line.split("\t")[0]
        itos.append(line.split("\t")[0])
        
class SPTokenizer(BaseTokenizer):
    "Wrapper around a SentncePiece tokenizer to make it a `BaseTokenizer`."
    def __init__(self, model_prefix:str):
        self.tok = spm.SentencePieceProcessor()
        self.tok.load(f'{model_prefix}.model')

    def tokenizer(self, t:str) -> List[str]:
#         return self.tok.EncodeAsIds(t)  ## tokenize + numericalize. We have them broken into two parts,just return tokens
        return self.tok.EncodeAsPieces(t)
        
class CustomTokenizer():
    '''Wrapper for SentencePiece toeknizer to fit into Fast.ai V1'''
    def __init__(self,tok_func:Callable,model_prefix:str, pre_rules:ListRules=None):
        self.tok_func,self.model_prefix = tok_func,model_prefix
        self.pre_rules  = ifnone(pre_rules,  defaults.text_pre_rules )
        
    def __repr__(self) -> str:
        res = f'Tokenizer {self.tok_func.__name__} using `{self.model_prefix}` model with the following rules:\n'
        for rule in self.pre_rules: res += f' - {rule.__name__}\n'
        return res        

    def process_text(self, t:str,tok:BaseTokenizer) -> List[str]:
        "Processe one text `t` with tokenizer `tok`."
        for rule in self.pre_rules: t = rule(t)  
        toks = tok.tokenizer(t)
    
        return toks 
    
    def _process_all_1(self,texts:Collection[str]) -> List[List[str]]:
        'Process a list of `texts` in one process'
        tok = self.tok_func(self.model_prefix)
        return [self.process_text(t,tok) for t in texts]
                                                                     
    def process_all(self, texts:Collection[str]) -> List[List[str]]: 
        "Process a list of `texts`."                                 
        return self._process_all_1(texts)

In [ ]:
itos[-10:]

In [ ]:
mycust_tok = CustomTokenizer(SPTokenizer,model_prefix,pre_rules=default_rules)
sp_vocab = Vocab(itos)

In [ ]:
data = TextClasDataBunch.from_df(PATH,df_train,df_valid,df_test,
                               tokenizer=mycust_tok, vocab=sp_vocab,
                               text_cols='new_text', label_cols='is_humor')

In [ ]:
data.show_batch(10)  #less xxunk, but still a fair amount...

### Check `xxunk` fraction on humor data

In [ ]:
pct_xxunk = pd.Series([sum(y==0)/len(y) for y in [x.data for x in data.train_dl.x]])
pct_xxunk[pct_xxunk>0].shape, pct_xxunk.shape

In [ ]:
498/20704.

In [ ]:
pd.Series([sum(y==0)/len(y) for y in [x.data for x in data.train_dl.x]]).plot()
_ = plt.title('Pct UNK in each tweet')

### build and train model

In [ ]:
config = tfmer_clas_config.copy()
config['n_layers'] = 8
config['ctx_len'] = 128
config['d_model'] = 512
config['d_inner'] = 2048
config['mask'] = True  

learn = text_classifier_learner(data, AWD_LSTM, drop_mult=0.5,pretrained=False,
                               config=config)
learn.load_encoder('twitter_es_enc_Tfm_0516')

In [ ]:
lr_find(learn)

In [ ]:
learn.recorder.plot(skip_end=10)

In [ ]:
## replace with attention head for classifier?
learn.freeze()
learn.fit_one_cycle(20, 1e-2, moms=(0.8,0.7),
                   callbacks=[SaveModelCallback(learn,every='improvement',mode='max',monitor='accuracy',name='best_acc_model')])

In [ ]:
learn.recorder.plot_losses()
learn.recorder.plot_metrics()

In [ ]:
learn.save('Tfm_trained_head')

In [ ]:
#learn = learn.load('rnn_trained_head')

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(20,slice(1e-3/(2.6**2),1e-3),moms=(0.7,0.4), wd=0.10,
                   callbacks=[SaveModelCallback(learn,every='improvement',mode='max',monitor='accuracy',name='best_acc_model')])

In [ ]:
learn.recorder.plot_losses()
learn.recorder.plot_metrics()

In [ ]:
learn.freeze_to(-3)

learn.fit_one_cycle(20,slice(1e-3/(2.6**3),1e-3),moms=(0.7,0.4), wd=0.10,
                   callbacks=[SaveModelCallback(learn,every='improvement',mode='max',monitor='accuracy',name='best_acc_model')])

In [ ]:
learn.recorder.plot_losses()
learn.recorder.plot_metrics()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(20,slice(1e-4/(2.6**4),1e-3),moms=(0.6,0.4), pct_start=0.10
                   callbacks=[SaveModelCallback(learn,every='improvement',mode='max',monitor='accuracy',name='best_acc_model')])

In [ ]:
learn.recorder.plot_losses()
learn.recorder.plot_metrics()

In [ ]:
preds_t,y_t,losses_t = learn.get_preds(DatasetType.Train,with_loss=True)
interp_t = ClassificationInterpretation(learn, preds_t, y_t, losses_t)
interp_t.plot_confusion_matrix()

preds,y,losses = learn.get_preds(with_loss=True)
interp = ClassificationInterpretation(learn, preds, y, losses)
interp.plot_confusion_matrix()

In [ ]:
from fastai.text.models.awd_lstm import TextClassificationInterpretation
from matplotlib.pyplot import cm
txt_ci = TextClassificationInterpretation.from_learner(learn)
(_,idxs) = interp.top_losses(5)
for idx in to_np(idxs):
    print("-"*10)
    txt_ci.show_intrinsic_attention(df_train.iloc[idx,1],cmap=cm.Blues) #textify breaks?

In [ ]:
learn.save("haha_20190516_tweet_vocab_qrnn")

### Save predictions for use in Ensemble method (not used)

In [ ]:
pd.DataFrame(to_np(preds)).to_csv('rnn_sent_preds.csv')

In [ ]:
## fastai "test" has no labels.  It is a true test, not a whitholding set!!  We need a second valid set?
## move the df_test into the "valid" position so we can run inference and get preds on randomized vals
data_t = TextClasDataBunch.from_df(PATH,df_train,df_test,
                               tokenizer=mycust_tok, vocab=sp_vocab,
                               text_cols='new_text', label_cols='is_humor')
learn.data = data_t
t_preds,t_y,t_losses = learn.get_preds(with_loss=True,ds_type=DatasetType.Valid)

In [ ]:
pd.DataFrame(to_np(t_preds)).to_csv('rnn_sent_t_preds.csv')